# Chapitre 4 : Validation de modèles

Email : <a href='mailto:madani.a@ucd.ac.ma'>madani.a@ucd.ac.ma</a>
<img src='images/robot.png'>

## 1. Introduction

<p>
Il est toujours nécessaire de valider la stabilité de votre modèle d'apprentissage automatique. Cela veut dire qu'il ne suffit pas d'adapter le modèle à vos données d'entraînement et espérer qu'il fonctionnerait avec précision pour les données réelles qu'il n'a jamais vues auparavant. 
</p>
<p>
Dans ce chapitre nous allons aborder le principe de validation de modèles dans le contexte de machine learning. Après une petite description des méthodes de validation les plus populaires, nous allons présenter des exemples d'implémentation en utilisant le langage Python.
</p>

## 2. Validation

<p>
Le processus consistant à déterminer si les résultats numériques quantifiant les relations entre les variables sont acceptables en tant que descriptions des données est appelé validation.

Dans ce processus, une estimation numérique de la différence entre les réponses prédites et les réponses originales est effectuée. Cette estimation est appelée erreur d'apprentissage. Elle nous permet d'avoir une idée sur la qualité de notre modèle. 
</p>

<p>
Plusieurs modèles de validation utilisés dans le contexte de machine learning sont proposés, mais nous n'allons aborder que quelques unes. Pour plus d'informations sur d'autres modèles, consultez <a href="http://scikit-learn.org/stable/modules/cross_validation.html">ce lien </a>
</p>
<p>
Dans la suite de ce chapitre, nous allons tester quelques approches de validation appliqués à l'algorithme KNN en utilisant le dataset Iris. On peut utiliser n'importe quel autre algorithme (SVM, Regression lineaire, ...).
</p>


Nous allons commencer par importer le dataset Iris que nous allons utiliser dans les exemples de ce chapitre

In [99]:
# Importer load_iris() permettant de charger le dataset Iris
from sklearn.datasets import load_iris
# Charger le dataset Iris
iris = load_iris()
# X : données
X = iris.data
# y : classes
y = iris.target
#Vérifier la structure de X et y
print(X.shape, y.shape)

(150, 4) (150,)


## 3. Exemples de modèles de validation

### 3.1 Entraîner et tester sur le Dataset entier

Cette procédure, appelée aussi approche <strong>naîve</strong>, consiste à :
<ul>
<li>Entraîner le modèle sur le dataset en entier
<li>Tester le modèle sur le même dataset, et évaluer à quel point nous avons bien comparé les valeurs de classess prédites avec les vraies valeurs de réponse.
</ul>

Commençons par choisir un modèle et ses hyperparamètres. Dans notre exemple, nous allons utiliser le classificateur k-neighbors avec n_neighbors = 5 (valeur par défaut).

In [100]:
from sklearn.neighbors import KNeighborsClassifier
# clf pour classifieur. Un classifieur est un modèle, tels que KNN, SVC, ...
clf = KNeighborsClassifier()

Ensuite, nous entraînons le modèle et nous l'utilisons pour prédire les classes des données que nous connaissons déjà :

In [101]:
# Entraînement du modèle
clf.fit(X, y)
# y_pred : y prédite
y_pred = clf.predict(X)

Enfin, nous calculons la fraction des points correctement étiquetés :

In [102]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

0.9666666666666667

### 3.2 Méthode Holdout (Train/test split)

Pour remèdier aux inconvénients de la méthode précédente, le principe de la méthode <strong>holdout</strong> consiste à supprimer une partie des données d'apprentissage et à l'utiliser pour obtenir des prédictions du modèle entraîné sur le reste des données.
</p>
<p>
L'ensemble d'apprentissage (l'ensemble d'entraînement) contient une sortie connue et le modèle apprend sur ces données afin d'être généralisé à d'autres données plus tard. Nous avons besoin, aussi, de l'ensemble de données de test afin de tester la prédiction de notre modèle sur ce sous-ensemble.
</p>
<img src="images/train_test_split1.png">
<p>
L'estimation de l'erreur indique ensuite comment notre modèle se comporte sur des données non vues auparavant. C'est un type simple de technique de validation croisée, également connu sous le nom de méthode <strong>train/test split</strong>.
</p>

Voyons maintenant comment faire cela en Python. Nous utilisons la bibliothèque <a href="http://scikit-learn.org/stable/index.html">Scikit-Learn</a> et plus particulièrement la méthode <a href="http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.

In [103]:
#Importer la fonction train_test_split
from sklearn.model_selection import train_test_split
# Diviser les données d'entraînement en deux sous ensembles Train et Test
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.2, shuffle=True)
# Entraîner le modèle sur l'ensemble d'entraînement
clf.fit(X_train, y_train)
# evaluer le modèle sur l'ensemble de test
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.875

<p>
Nous avons utilisé la fonction <strong>train_test_split</strong> pour faire la division (le split). Le test_size à l'intérieur de la fonction indique le pourcentage de données à conserver pour le test. C'est généralement autour de 80/20 ou 70/30.
</p>
<p>
Comme on peut le voir ici, nous avons obtenu un résultat plus raisonnable.
</p>

### 3.3 Cross-Validation (Validation croisée)

#### 3.3.1 Two-Folds

L'un des inconvénients de l'utilisation de la méthode holdout pour la validation de modèles est que nous avons retiré une partie de nos données de l'ensemble de données destinées à l'entraînement du modèle. Dans l'exemple du cas précédent, 20% de l'ensemble de données ne va pas contribuer à l'entraînement du modèle! Ce n'est pas optimal et peut causer des problèmes, en particulier si l'ensemble initial de données d'entraînement est petit.

Une façon de remèdier à ce problème est d'utiliser <strong>Cross-Validation</strong> (la validation croisée). L'idée est de construire plusieurs sous-ensemble (Folds) où chaque sous-ensemble de données est utilisé à la fois comme un ensemble d'apprentissage et comme un ensemble de validation. Visuellement, cela pourrait ressembler à la figure suivante.

<img src="images/two-folds.png">

Ici, nous procédons à deux tests de validation, en utilisant alternativement chaque moitié des données comme dans le cas de la méthode holdout. En utilisant les données importées auparavant, nous pourrions l'implémenter comme ceci :

In [114]:
import numpy as np
#Importer la fonction train_test_split
from sklearn.model_selection import train_test_split
# Diviser les données d'entraînement en deux sous ensembles Fold1 et Fold2 ayant 50% chacun
X1, X2, y1, y2 = train_test_split(X, y,train_size=0.5)
# Entraîner le modèle sur le premier Fold
clf.fit(X1, y1)
# evaluer le modèle sur le deuxième Fold
y_pred2 = clf.predict(X2)
# Entraîner le modèle sur le deuxième Fold
clf.fit(X2, y2)
# evaluer le modèle sur le premier Fold
y_pred1 = clf.predict(X1)
#Afficher la précision
score1 = accuracy_score(y1, y_pred1)
score2 = accuracy_score(y2, y_pred2)
score = (score1 + score2)/2
print("Score 1 :", score1, "Score 2 :", score2, " Score final :",score)


Score 1 : 0.96 Score 2 : 0.9466666666666667  Score final : 0.9533333333333334


Nous obtenons deux scores de précision, que nous pourrions combiner (par exemple, en prenant la moyenne) pour obtenir une meilleure mesure de la performance du modèle global. Cette forme particulière de validation croisée est une double validation croisée dans laquelle nous avons divisé les données en deux ensembles et utilisés chacun à son tour comme un ensemble de validation.

#### 3.3.2 K-Folds

Nous pourrions développer l'idée précédente pour utiliser encore plus de tests, et plus de split dans les données. Par exemple, la Figure suivante est une représentation visuelle de la validation croisée contenant k folds (sous-ensembles).

<img src="images/k-folds.png">

<p>
Dans l'exemple suivant, nous divisons les données en 5 groupes (folds), et nous utilisons chacun d'eux à tour de rôle pour évaluer le modèle sur l'autre 4/5 des données.
</p>
<p>Ce serait très fastidieux de l'implémenter manuellement. Nous pouvons utiliser la fonction <strong>cross_val_score</strong> de Scikit-Learn pour le faire succinctement :
</p>

In [105]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, y, cv=5)
print("Tous les scores :", scores)
print("Le score final :", scores.mean())

Tous les scores : [0.96666667 1.         0.93333333 0.96666667 1.        ]
Le score final : 0.9733333333333334


La répétition de la validation entre les différents sous-ensembles de données nous donne une meilleure idée de la performance de l'algorithme.

#### 3.3.3 Leave-One-Out-Cross-Validation (LOOCV)

Scikit-Learn met en œuvre un certain nombre de schémas de validation croisée qui sont utiles dans des situations particulières. Ces schémas sont implémentés via des itérateurs dans le module model_selection. Par exemple, nous pourrions vouloir aller au cas extrême où notre nombre de splits est égal au nombre d'exemples du dataset ; c'est-à-dire que nous entraînons notre modèle sur toutes les données sauf une à chaque itération. Ce type de validation croisée peut être utilisé comme suit:

In [118]:
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(clf, X, y, cv=LeaveOneOut())
print("Tous les score :",scores)
print("Score final :",scores.mean())


Tous les score : [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]
Score final : 0.9666666666666667


Étant donné que nous avons 150 échantillons, la validation croisée «Leave One Out Cross Validation» donne des résultats pour 150 tets, et le score indique une prédiction réussie (1.0) ou infructueuse (0.0). La moyenne de ceux-ci nous donne une estimation du taux d'erreur.

## 4. Finetuning avec GridSearchCV

La recherche par grille avec validation croisée (GridSearchCV) est une méthode d'optimisation des hyperparamètres qui explore de manière systématique un espace spécifié d'hyperparamètres, évaluant la performance d'un modèle à l'aide de la validation croisée. C'est une manière de trouver la meilleure combinaison d'hyperparamètres pour un modèle donné et un ensemble de données. Voici les étapes :

### 4.1. Importer les bibliothèques nécessaires :

Vous commencez généralement par importer les bibliothèques nécessaires de scikit-learn :

In [107]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

Remplacez **KNeighborsClassifier** par la classe de modèle scikit-learn que vous souhaitez utiliser, telle que **SVM** pour les machines à vecteurs de support, **RandomForestClassifier** pour les forêts aléatoires, etc.

### 4.2. Charger et diviser les données :

In [108]:
# Chargez votre jeu de données
iris = load_iris()
X = iris.data
y = iris.target
# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 4.3. Définir le modèle et la grille de paramètres :

In [109]:
# Créez une instance de votre modèle, KNN dans notre cas. Vous pouvez remplacer
#KNN par n'importe quel autre calssifieur : SVM, ...
modele = KNeighborsClassifier()

# Définissez la grille de paramètres à rechercher
param_grid = {
    'n_neighbors': range(3,30,2),
    #'p': [1, 2],  # 1 for Manhattan distance, 2 for Euclidean distance
    'metric':['euclidean','manhattan','l1','l2','cosine','minkowski']
}

Vouds pouvez ajouter d'autres paramètres

### 4.4. Créer l'objet GridSearchCV :

In [110]:
grid_search = GridSearchCV(modele, param_grid, cv=5, scoring='accuracy')

- cv : Nombre de plis de validation croisée.
- scoring : La métrique d'évaluation à optimiser. Les choix courants incluent 'accuracy', 'precision', 'recall', 'f1', etc.

### 4.5. Ajuster le modèle aux données :

In [111]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'l1', 'l2',
                                    'cosine', 'minkowski'],
                         'n_neighbors': range(3, 30, 2)},
             scoring='accuracy')

### 4.6. Accéder aux meilleurs paramètres et au meilleur modèle :

In [112]:
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur score : {:.2f}".format(grid_search.best_score_))

meilleur_modele = grid_search.best_estimator_

Meilleurs paramètres :  {'metric': 'cosine', 'n_neighbors': 21}
Meilleur score : 0.99


Après l'ajustement, vous pouvez accéder aux meilleurs paramètres avec grid_search.best_params_, au meilleur score avec grid_search.best_score_, et au meilleur modèle avec grid_search.best_estimator_.

### 4.7. Évaluer sur l'ensemble de test :

In [113]:
precision_test = meilleur_modele.score(X_test, y_test)
print("Précision sur le test : {:.2f}%".format(precision_test * 100))

Précision sur le test : 93.33%


Ce processus vous aide à trouver la combinaison d'hyperparamètres qui donne la meilleure performance sur votre ensemble de données tout en évitant le surajustement aux données d'entraînement. Ajustez la classe du modèle, la grille des paramètres et l'ensemble de données en fonction de votre cas d'utilisation spécifique.